In [ ]:
using ITensors
using Plots
using Plots.PlotMeasures
using Parameters
using Statistics

### Build Geometry

In [ ]:
Sites = Vector{<:Index}

In [ ]:
# Build 1D chain
# using the Lattice vector is overkill for a chain but makes it easy to
# generalize to other geometries
function chain_nearest_lattice(n::Int; is_periodic = false, neighbor = 1)
  nbonds = n - (is_periodic ? 0 : neighbor)
  chain = Lattice(undef, nbonds)
  for i in 1:nbonds
    chain[i] = LatticeBond(i, ((i + neighbor - 1) % n) + 1)
  end
  chain
end

In [ ]:
@with_kw struct ChainParams
  n::Int
  is_periodic::Bool = false
end

### Build the Heisenberg Hamiltonian

$$ H = J_1\sum_i S_i \cdot S_{i + 1} + h\sum_i S_i^z = J_1\bigg(\sum_i S_i^zS_{i+1}^z + \frac{1}{2}S_i^+S_{i+1}^- + \frac{1}{2}S_i^-S_{i+1}^+\bigg) + h\sum_i S_i^z$$

In [ ]:
function build_heisenberg_hamiltonian(sites::Sites, chainparams::ChainParams, J1::Real, J2::Real, h::Real)

    nearest = chain_nearest_lattice(chainparams.n, is_periodic=chainparams.is_periodic, neighbor=1)
    next_nearest = chain_nearest_lattice(chainparams.n, is_periodic=chainparams.is_periodic, neighbor=2)

    ampo = OpSum()

    for b in nearest
        ampo +=       J1, "Sz", b.s1, "Sz", b.s2
        ampo += 0.5 * J1, "S+", b.s1, "S-", b.s2
        ampo += 0.5 * J1, "S-", b.s1, "S+", b.s2
    end

    for b in next_nearest
        ampo +=       J2, "Sz", b.s1, "Sz", b.s2
        ampo += 0.5 * J2, "S+", b.s1, "S-", b.s2
        ampo += 0.5 * J2, "S-", b.s1, "S+", b.s2
    end

    for i in 1:length(sites)
        ampo += h, "Sz", i
    end

    return MPO(ampo, sites)
end

### Build the Ising Hamiltonian

$$ H = -J(\sum_i S^z_i \cdot S^z_{i + 1} + g\sum_i S^x_i)$$

In [ ]:
function build_ising_hamiltonian(sites, chainparams::ChainParams, J::Real, g::Real)
  
  chain = chain_nearest_lattice(chainparams.n, is_periodic=chainparams.is_periodic, neighbor=1)

  ampo = OpSum()

  for b in chain
      ampo += -J, "Sz", b.s1, "Sz", b.s2
      ampo += -g * J, "Sx", b.s1
  end

  return MPO(ampo, sites)
end

### Run DMRG

In [ ]:
@with_kw struct SweepParams
  num::Int
  dims::Array{Int}
  cutoff::Real
end

@with_kw struct DMRGParams
  initial_bond_dimension::Int
  sweep::SweepParams
end

function run_dmrg(sites, H, ψ₀, params::DMRGParams)
  sweeps = Sweeps(params.sweep.num)
  setmaxdim!(sweeps, params.sweep.dims...)
  setcutoff!(sweeps, params.sweep.cutoff)

  return dmrg(H, ψ₀, sweeps);
end

In [ ]:
"""
Pattern for a parameterized type which allows making literals into types. 
This allows for new types to be defined across the codebase and leveraging 
multiple dispatch.
"""

struct InitialStateType{T}
end

InitialStateType(s::AbstractString) = InitialStateType{Symbol(s)}()

macro InitialStateType_str(s)
  return InitialStateType{Symbol(s)}
end

function build_initial_state(t::InitialStateType"alt", sites)
  state = [isodd(n) ? "Up" : "Dn" for n=1:length(sites)]
  return productMPS(sites, state)
end

function build_initial_state(t::InitialStateType"all up", sites)
  state = fill("Up", length(sites))
  return productMPS(sites, state)
end

function build_initial_state(t::InitialStateType"random", sites)
  return randomMPS(sites, )
end

build_initial_state(s::AbstractString, sites) = build_initial_state(InitialStateType(s), sites)

In [ ]:
@with_kw struct HeisenbergParams
  chainparams::ChainParams
  dmrgparams::DMRGParams
  sitetype::String
  J1::Real
  J2::Real = 0
  h::Real = 0
  initial_state_type::String = "alt"
end

function make_observations(ψ, params::HeisenbergParams)
  n = params.chainparams.n

  # Sz expectation on each site
  szs = expect(ψ, "Sz")
  szexpplot = plot(szs, title = "S_z expectation")
  ylims!(-0.6, 0.6)
  xlabel!("Site")
  ylabel!("<S_z>")

  # Sz correlation
  cm = correlation_matrix(ψ, "Sz", "Sz")
  qrt = floor(Integer, n/4)
  szcorrplot = plot(abs.(cm[qrt, qrt:(3*qrt)]),  title = "S_z correlation")
  xlabel!("Site")

  println("\nN = $n, J1 = $(params.J1), J2 = $(params.J2)")
  p = plot(szexpplot, szcorrplot, layout = (2, 1), titlefontsize=11, legend=false)
  display(p)
  
  return nothing
end

function run_heisenberg_model(params:: HeisenbergParams; makeobs=true)
  @unpack chainparams, dmrgparams, sitetype, J1, J2, h, initial_state_type = params

  sites = siteinds(sitetype, chainparams.n; conserve_qns = true)

  H = build_heisenberg_hamiltonian(sites, chainparams, J1, J2, h)

  ψ₀ =  build_initial_state(initial_state_type, sites)
  energy, ψ = run_dmrg(sites, H, ψ₀, dmrgparams)
  
  if makeobs
    make_observations(ψ, params)
  end
  
  return ψ
end

In [ ]:
@with_kw struct IsingParams
  chainparams::ChainParams
  dmrgparams::DMRGParams
  sitetype::String
  J::Real
  g::Real = 0
  initial_state_type::String = "alt"
end

function make_observations(ψ, params::IsingParams)
  @unpack chainparams, dmrgparams, sitetype, J, g, initial_state_type = params
  @unpack n = chainparams

  # Sz expectation on each site
  szs = expect(ψ, "Sz")
  szexpplot = plot(szs, title = "S_z expectation")
  ylims!(-0.6, 0.6)
  xlabel!("Site")
  ylabel!("<S_z>")

  # Sz correlation
  cm = correlation_matrix(ψ, "Sz", "Sz")
  qrt = floor(Integer, n/4)
  szcorrplot = plot(abs.(cm[qrt, qrt:(3*qrt)]),  title = "S_z correlation")
  xlabel!("Site")

  println("\nN = $n, J = $J, J2 = $g")
  p = plot(szexpplot, szcorrplot, layout = (2, 1), titlefontsize=11, legend=false)
  display(p)
  
  return nothing
end

function run_ising_model(params::IsingParams; makeobs=true)
  @unpack chainparams, dmrgparams, sitetype, J, g, initial_state_type = params

  sites = siteinds(sitetype, chainparams.n)

  H = build_ising_hamiltonian(sites, chainparams, J, g)

  ψ₀ =  build_initial_state(initial_state_type, sites)
  energy, ψ = run_dmrg(sites, H, ψ₀, dmrgparams)
  
  if makeobs
    make_observations(ψ, params)
  end
  
  return ψ
end

# Heisenberg Model

In [ ]:
chainparams = ChainParams(n = 5, is_periodic=false)
sweepparams = SweepParams(num = 10, dims = [5, 10, 20, 100, 100], cutoff = 1E-8)
dmrgparams = DMRGParams(initial_bond_dimension = 10, sweep=sweepparams)
heisenbergparams = HeisenbergParams(sitetype = "S=1/2", J1 = 1, chainparams = chainparams, dmrgparams = dmrgparams)

run_heisenberg_model(heisenbergparams);

In [ ]:
chainparams = ChainParams(n = 50)
sweepparams = SweepParams(num = 10, dims = [5, 10, 20, 100, 100], cutoff = 1E-8)
dmrgparams = DMRGParams(initial_bond_dimension = 10, sweep=sweepparams)
heisenbergparams = HeisenbergParams(sitetype = "S=1/2", J1 = 1, chainparams = chainparams, dmrgparams = dmrgparams)

run_heisenberg_model(heisenbergparams);

In [ ]:
chainparams = ChainParams(n = 100)
sweepparams = SweepParams(num = 10, dims = [5, 10, 20, 100, 100], cutoff = 1E-8)
dmrgparams = DMRGParams(initial_bond_dimension = 10, sweep=sweepparams)
heisenbergparams = HeisenbergParams(sitetype = "S=1/2", J1 = 1, J2=-1, chainparams = chainparams, dmrgparams = dmrgparams)

run_heisenberg_model(heisenbergparams);

# Ising Model

In [ ]:
chainparams = ChainParams(n = 5, is_periodic=false)
sweepparams = SweepParams(num = 10, dims = [5, 10, 20, 100, 100], cutoff = 1E-8)
dmrgparams = DMRGParams(initial_bond_dimension = 10, sweep=sweepparams)
isingparams = IsingParams(sitetype="S=1/2", J=1, g=0, chainparams = chainparams, dmrgparams = dmrgparams)

run_ising_model(isingparams);

In [ ]:
chainparams = ChainParams(n = 100, is_periodic=false)
sweepparams = SweepParams(num = 10, dims = [5, 10, 20, 100, 100], cutoff = 1E-8)
dmrgparams = DMRGParams(initial_bond_dimension = 10, sweep=sweepparams)
isingparams = IsingParams(sitetype="S=1/2", J=1, g=0.1, chainparams = chainparams, dmrgparams = dmrgparams)

run_ising_model(isingparams);

In [ ]:
chainparams = ChainParams(n = 100, is_periodic=false)
sweepparams = SweepParams(num = 10, dims = [5, 10, 20, 100, 100], cutoff = 1E-8)
dmrgparams = DMRGParams(initial_bond_dimension = 10, sweep=sweepparams)
isingparams = IsingParams(sitetype="S=1/2", J=1, g=0.5, chainparams = chainparams, dmrgparams = dmrgparams)

run_ising_model(isingparams);

In [ ]:
chainparams = ChainParams(n = 100, is_periodic=false)
sweepparams = SweepParams(num = 10, dims = [5, 10, 20, 100, 100], cutoff = 1E-8)
dmrgparams = DMRGParams(initial_bond_dimension = 10, sweep=sweepparams)
isingparams = IsingParams(sitetype="S=1/2", J=1, g=1, chainparams = chainparams, dmrgparams = dmrgparams)

run_ising_model(isingparams);

In [ ]:
chainparams = ChainParams(n = 100, is_periodic=false)
sweepparams = SweepParams(num = 10, dims = [5, 10, 20, 100, 100], cutoff = 1E-8)
dmrgparams = DMRGParams(initial_bond_dimension = 10, sweep=sweepparams)
isingparams = IsingParams(sitetype="S=1/2", J=1, g=0.1, chainparams = chainparams, dmrgparams = dmrgparams)

run_ising_model(isingparams);

In [ ]:
chainparams = ChainParams(n = 100, is_periodic=false)
sweepparams = SweepParams(num = 10, dims = [5, 10, 20, 100, 100], cutoff = 1E-8)
dmrgparams = DMRGParams(initial_bond_dimension = 10, sweep=sweepparams)
isingparams = IsingParams(sitetype="S=1/2", J=-1, g=0.1, chainparams = chainparams, dmrgparams = dmrgparams)

run_ising_model(isingparams);

## Sweep g/J to see phase transition

In [ ]:
chainparams = ChainParams(n = 50, is_periodic=false)
sweepparams = SweepParams(num = 12, dims = [5, 10, 20, 100, 200], cutoff = 1E-8)
dmrgparams = DMRGParams(initial_bond_dimension = 10, sweep=sweepparams)

gsamples = 0:0.01:1
sz_samples = zeros(length(gsamples))
for (idx, g) in enumerate(gsamples)
  isingparams = IsingParams(sitetype="S=1/2", J=1, g=g, chainparams=chainparams, dmrgparams=dmrgparams)

  ψ = run_ising_model(isingparams, makeobs=false);
  szs = expect(ψ, "Sz")

  # # take average of middle half of chain
  len = length(szs)
  midszs = szs[(len ÷ 4):(3 * len ÷ 4)]
  mean_sz = mean(abs.(midszs))
  # mid = abs(szs[length(szs) ÷ 2])
  sz_samples[idx] = mean_sz
end

In [ ]:

plot(gsamples, sz_samples, legend=false, title="Sz Expectation vs Transverse Field Strength")
xlabel!("g/J")
ylabel!("<Sz>")